In [1]:
import pandas as pd

In [ ]:
# Part 1
# Two Kinds of one-hot encoding

df = pd.DataFrame([
            ['green', 'M', 10.1, 'class1'], 
            ['red', 'L', 13.5, 'class2'], 
            ['blue', 'XL', 15.3, 'class1']])

In [2]:
df.columns = ['color', 'size', 'prize', 'class label']

In [3]:
# Firstly, encode the features (size, class) with relative order
size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}
df['size'] = df['size'].map(size_mapping)

In [5]:
class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}
df['class label'] = df['class label'].map(class_mapping)
df

,color,size,prize,class label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [8]:
#Secondly, encode the other no order one-hot features (color)
pd.get_dummies(df, columns=["color"])

,size,prize,class label,color_blue,color_green,color_red
0,1,10.1,0,0,1,0
1,2,13.5,1,0,0,1
2,3,15.3,0,1,0,0


In [ ]:
#Part 2 
#Pandas.get_dummies().groupby()

In [9]:
df = pd.DataFrame([
            ['A', 'id1', 'gs1'], 
            ['A', 'id2', 'gs2'], 
            ['B', 'id1', 'gs1'],
            ['B', 'id2', 'gs2'],
            ['C', 'id1', 'gs1'],
            ['C', 'id1', 'gs2']
])
df.columns = ['event_code', 'installation_id', 'game_session','timestamp']

In [17]:
join_one = pd.get_dummies(df[['event_code','installation_id',"game_session"]],
                         columns=['event_code']).groupby(['installation_id',"game_session"],
                        as_index=False,sort=False).agg(sum)
join_one

,installation_id,game_session,event_code_A,event_code_B,event_code_C
0,id1,gs1,1,1,1
1,id2,gs2,1,1,0
2,id1,gs2,0,0,1


In [97]:
df=pd.read_csv('../data/t.csv',
                         usecols=['installation_id','game_session','event_code',
                                  'event_count','game_time','event_id','timestamp',
                                  'type','world','title'])

In [163]:
join_one=pd.get_dummies(df[['event_code','installation_id','game_session']],
                            columns=['event_code']).groupby(['installation_id','game_session'],
                                                            as_index=False,sort=False).sum()
join_one.head()

,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,...,event_code_4080,event_code_4090,event_code_4095,event_code_4100,event_code_4110,event_code_4220,event_code_4230,event_code_4235,event_code_5000,event_code_5010
0,0001e90f,45bb1e1b6b50c07b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0001e90f,17eeb7f223665f53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0001e90f,0848ef14a8dc6892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0001e90f,a1ec58f109218255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0001e90f,f11eb823348bfa23,1.0,0.0,57.0,0.0,57.0,0.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
agg={'event_count':['sum','mean'],'game_time':['sum','mean'],'event_id':'count'}
agg

{'event_count': ['sum', 'mean'],
 'game_time': ['sum', 'mean'],
 'event_id': 'count'}

In [100]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    return df

In [101]:
time_features=['month','hour','year','dayofweek','weekofyear']

In [102]:
df=extract_time_features(df)

In [111]:
join_two=df.drop(time_features,axis=1).groupby(['installation_id','game_session']
                                                   ,as_index=False,sort=False).agg(agg)
join_two.head()                   

installation_id      game_session event_count         game_time  \
                                            sum   mean        sum   
0        0001e90f  45bb1e1b6b50c07b           1    1.0          0   
1        0001e90f  17eeb7f223665f53           1    1.0          0   
2        0001e90f  0848ef14a8dc6892       35778  134.0   21135461   
3        0001e90f  a1ec58f109218255        4371   47.0    3349578   
4        0001e90f  f11eb823348bfa23      274911  371.0  225861954   

                 event_id  
            mean    count  
0       0.000000        1  
1       0.000000        1  
2   79159.029963      267  
3   36016.967742       93  
4  304806.955466      741

In [112]:
join_three=df[['installation_id','game_session','type','world','title']].groupby(
                ['installation_id','game_session'],as_index=False,sort=False).first()
join_three.head()

,installation_id,game_session,type,world,title
0,0001e90f,45bb1e1b6b50c07b,Clip,NONE,Welcome to Lost Lagoon!
1,0001e90f,17eeb7f223665f53,Clip,MAGMAPEAK,Magma Peak - Level 1
2,0001e90f,0848ef14a8dc6892,Activity,MAGMAPEAK,Sandcastle Builder (Activity)
3,0001e90f,a1ec58f109218255,Activity,MAGMAPEAK,Sandcastle Builder (Activity)
4,0001e90f,f11eb823348bfa23,Game,MAGMAPEAK,Scrub-A-Dub


In [114]:
from scipy.stats import mode

In [144]:
join_four=df[time_features+['installation_id','game_session']].groupby(['installation_id',
                'game_session'],as_index=False,sort=False).agg(mode)[time_features].applymap(lambda x: x[0][0])
join_four.head()

,month,hour,year,dayofweek,weekofyear
0,9,17,2019,4,36
1,9,17,2019,4,36
2,9,17,2019,4,36
3,9,17,2019,4,36
4,9,18,2019,4,36


In [148]:
join_one.head()

,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,...,event_code_4220,event_code_4230,event_code_4235,event_code_5000,event_code_5010,month,hour,year,dayofweek,weekofyear
0,0001e90f,45bb1e1b6b50c07b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
1,0001e90f,17eeb7f223665f53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
2,0001e90f,0848ef14a8dc6892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
3,0001e90f,a1ec58f109218255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
4,0001e90f,f11eb823348bfa23,1.0,0.0,57.0,0.0,57.0,0.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,9,18,2019,4,36


In [159]:
join_one = join_one.drop(time_features,axis=1)
join_one.head()

,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,...,event_code_4080,event_code_4090,event_code_4095,event_code_4100,event_code_4110,event_code_4220,event_code_4230,event_code_4235,event_code_5000,event_code_5010
0,0001e90f,45bb1e1b6b50c07b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0001e90f,17eeb7f223665f53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0001e90f,0848ef14a8dc6892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0001e90f,a1ec58f109218255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0001e90f,f11eb823348bfa23,1.0,0.0,57.0,0.0,57.0,0.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
join_one = join_one.join(join_four)
join_one.head()
#time_features=['month','hour','year','dayofweek','weekofyear']

,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,...,event_code_4220,event_code_4230,event_code_4235,event_code_5000,event_code_5010,month,hour,year,dayofweek,weekofyear
0,0001e90f,45bb1e1b6b50c07b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
1,0001e90f,17eeb7f223665f53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
2,0001e90f,0848ef14a8dc6892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
3,0001e90f,a1ec58f109218255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9,17,2019,4,36
4,0001e90f,f11eb823348bfa23,1.0,0.0,57.0,0.0,57.0,0.0,15.0,15.0,...,0.0,0.0,0.0,0.0,0.0,9,18,2019,4,36


In [162]:
join_five=(join_one.join(join_two.drop(['installation_id','game_session'],axis=1))). \
                        join(join_three.drop(['installation_id','game_session'],axis=1))
join_five.head()

,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_2040,event_code_2050,...,dayofweek,weekofyear,"(event_count, sum)","(event_count, mean)","(game_time, sum)","(game_time, mean)","(event_id, count)",type,world,title
0,0001e90f,45bb1e1b6b50c07b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,36,1,1.0,0,0.000000,1,Clip,NONE,Welcome to Lost Lagoon!
1,0001e90f,17eeb7f223665f53,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,36,1,1.0,0,0.000000,1,Clip,MAGMAPEAK,Magma Peak - Level 1
2,0001e90f,0848ef14a8dc6892,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,36,35778,134.0,21135461,79159.029963,267,Activity,MAGMAPEAK,Sandcastle Builder (Activity)
3,0001e90f,a1ec58f109218255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,36,4371,47.0,3349578,36016.967742,93,Activity,MAGMAPEAK,Sandcastle Builder (Activity)
4,0001e90f,f11eb823348bfa23,1.0,0.0,57.0,0.0,57.0,0.0,15.0,15.0,...,4,36,274911,371.0,225861954,304806.955466,741,Game,MAGMAPEAK,Scrub-A-Dub


In [ ]:
# groupby.first()
df = pd.DataFrame({'id' : [2,1,1,1,2,3,3,3,3,4,4,5,6,6,6,7,7],
                'value'  : ["first","second","second","first",
                            "second","first","third","fourth",
                            "fifth","second","fifth","first",
                            "first","second","third","fourth","fifth"]})
df

In [93]:
df.groupby('id')

In [94]:
df.groupby('id').first()

,value
id,
1,second
2,first
3,first
4,second
5,first
6,first
7,fourth
